In [ ]:
import pandas as pd\nimport numpy as np\nimport yfinance as yf\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom pypfopt import expected_returns, risk_models, EfficientFrontier, plotting

In [ ]:
funds = pd.read_csv('../data/bbva_funds.csv')\nfunds

In [ ]:
tickers = funds['Proxy_Ticker'].tolist()\ndata = yf.download(tickers, start='2020-01-01', progress=False)['Adj Close'].dropna()\ndata.tail()

In [ ]:
returns = data.pct_change().dropna()\nreturns.head()

In [ ]:
summary = pd.DataFrame({\n    'Annual Return': returns.mean() * 252,\n    'Volatility': returns.std() * np.sqrt(252)\n})\nsummary['Sharpe (rf=0)'] = summary['Annual Return'] / summary['Volatility']\nsummary

In [ ]:
plt.figure(figsize=(6,4))\nsns.heatmap(returns.corr(), annot=True, cmap='coolwarm', fmt='.2f')\nplt.title('Correlation Matrix')\nplt.show()

In [ ]:
mu = expected_returns.mean_historical_return(data)\nS = risk_models.sample_cov(data)

In [ ]:
ef = EfficientFrontier(mu, S)\nweights = ef.max_sharpe()\ncleaned = ef.clean_weights()\ncleaned

In [ ]:
ef.portfolio_performance(verbose=True)

In [ ]:
current = funds.set_index('Proxy_Ticker')['Weight_%'].to_dict()\ncomparison = pd.DataFrame({\n    'Current': current,\n    'Optimized': cleaned\n}).fillna(0)\ncomparison['Difference'] = comparison['Optimized'] - comparison['Current']\ncomparison

In [ ]:
ef_plot = EfficientFrontier(mu, S)\nfig, ax = plt.subplots(figsize=(8,6))\nplotting.plot_efficient_frontier(ef_plot, ax=ax, show_assets=False)\nret_opt, vol_opt, _ = ef.portfolio_performance()\nax.scatter(vol_opt, ret_opt, marker='*', s=150, c='orange', label='Optimized')\nw_cur = np.array(list(current.values()))\nret_cur = np.dot(w_cur, mu)\nvol_cur = np.sqrt(np.dot(w_cur.T, np.dot(S, w_cur)))\nax.scatter(vol_cur, ret_cur, marker='o', s=80, c='blue', label='Current')\nax.set_title('Efficient Frontier: Current vs Optimized')\nax.set_xlabel('Volatility (Risk)')\nax.set_ylabel('Expected Return')\nax.legend()\nplt.show()